In [1]:
prompt="Write a poem about dogs"

In [2]:
from lightweight import lightweight

lightweight(prompt)

/Users/jackson.makl@dataiku.com/agent/env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model facebook/opt-125m on mps


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Response: uit.
Well, we are on the verge of that.
We need to make it happen.
Loading model facebook/opt-125m on mps
Response: dogs.
I think we should do that. I know there's a lot of people who love dogs


"dogs.\nI think we should do that. I know there's a lot of people who love dogs"

In [3]:
lightweight(prompt,model_name="facebook/opt-350m")


Loading model facebook/opt-350m on mps
Response: dogs.
That's what i was thinking. I have a dog and I don't think I could


"dogs.\nThat's what i was thinking. I have a dog and I don't think I could"

In [4]:
lightweight(prompt,model_name="microsoft/phi-1_5")


Loading model microsoft/phi-1_5 on mps
Response: using a specific rhyme scheme.

Answer: 

Paws and tails, they


'using a specific rhyme scheme.\n\nAnswer: \n\nPaws and tails, they'

In [5]:
lightweight(prompt,model_name="TinyLlama/TinyLlama-1.1B-Chat-v0.6")

Loading model TinyLlama/TinyLlama-1.1B-Chat-v0.6 on mps
Response: dogs. 5. 4. A poem about a pet dog. 6. 5. A poem about a family pet that has passed away. 7. 6. A poem about a poodle. 8. 7. A poem about a family pet that has had a significant impact on your life. 9. 8. A poem about a pet that has been injured. 10. 9. A poem about a pet that has had a health issue. Remember to use specific examples, vivid imagery, and thought-provoking language to create a compelling and memorable poem about your pet dog.</s>


'dogs. 5. 4. A poem about a pet dog. 6. 5. A poem about a family pet that has passed away. 7. 6. A poem about a poodle. 8. 7. A poem about a family pet that has had a significant impact on your life. 9. 8. A poem about a pet that has been injured. 10. 9. A poem about a pet that has had a health issue. Remember to use specific examples, vivid imagery, and thought-provoking language to create a compelling and memorable poem about your pet dog.</s>'

In [6]:
lightweight(prompt,model_name="microsoft/Phi-4-mini-instruct")

Loading model microsoft/Phi-4-mini-instruct on mps


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Loading checkpoint shards: 100%|██████████| 2/2 [00:18<00:00,  9.44s/it]


Response: . The word "bark" should appear at least 5 times. Your response must have exactly


'. The word "bark" should appear at least 5 times. Your response must have exactly'

In [8]:
# lightweight(prompt,model_name="")